# XML reports to Elasticsearch

Functionality can be tested with the MEANTIME corpus [download](http://www.newsreader-project.eu/results/data/wikinews/)

Install dependencies and import the nlp_datau package

In [1]:
!pip install -r ./../requirements.txt
import sys
sys.path.append("./../")

You should consider upgrading via the '/Users/sanderputs/IdeaProjects/Research/venv/bin/python -m pip install --upgrade pip' command.


Configure the directory containing the xml files and xml document (id of the document in the xml file (using a dot notation)

In [2]:
directory_in_str = '/home/jovyan/work/E/Sander/verslagen/'
directory_in_str = '/home/jovyan/work/D/work/ACR'
directory_in_str = '/Users/sanderputs/git/nlp-datau/resources/data_ignored/report_test'

field_id = "ClinicalDocument.setId.@extension"

Defined a name for the Elasticsearch index (hint project name)

In [3]:
es_host = "doccano-dataset-tools-es01"
es_host =  "localhost"
es_index = "free-text-reports"

Configure whitelist dictionary: the key is the Elasticsearch document target field, the value the source field in the XML file.

The example below shows how to index field embedded field "text" to "text".
By definining an optional start-and/or-stop-line, you can extract substrings from the source field

In [4]:
whitelist_dict = {
                     "text": {
                         "field": "ClinicalDocument.component.structuredBody.component.section.text",
                         "type": "xml",
                         "xml-field": "getReportTextByAccessionnrResult.REPORT.REPORT_BODY.TEXT",
                         "start-line": "Verslagstatus:",
                         "stop-line": "Met collegiale groet,"
                     },
                    "R_DATELASTCHANGED": {
                         "field": "ClinicalDocument.component.structuredBody.component.section.text",
                         "type": "xml",
                         "xml-field": "getReportTextByAccessionnrResult.REPORT.REPORT_DATA.R_DATELASTCHANGED"
                     },
                    "patient_id": {
                        "field": "ClinicalDocument.recordTarget.patientRole.id",
                        "type": "list",
                        "key": "@extension",
                        "match-key": "@root",
                        "match-value": "2.16.840.1.113883.2.4.3.74.3.1.9.3",
                    }

                 }

An yaml configuration file can be used to store configuration (optional)

In [5]:
use_config_file = False
config_file = 'config/xml_to_excel_config.yaml'

import yaml
if use_config_file:
    with open(config_file, 'r') as yaml_file:
        cfg = yaml.safe_load(yaml_file)
        print(cfg)
        directory_in_str = cfg['directory_in_str']
        extension = cfg['extension']
        field_id = cfg['field_id']
        whitelist_dict = cfg['whitelist_dict']



Index directory

In [6]:
from nlp_datau.xml_to_index import XmlToIndex
XmlToIndex(es_host=es_host).iterate_xml_dir(
    directory_in_str,
            extension=".xml",
            es_index=es_index,
            field_id=field_id,
            whitelist_dict=whitelist_dict)

INFO:python-processor:start iteration /Users/sanderputs/git/nlp-datau/resources/data_ignored/report_test
INFO:python-processor:iteration finished /Users/sanderputs/git/nlp-datau/resources/data_ignored/report_test


Validate the number of documents at
[http://localhost:9200/[ES_INDEX]/_count](http://localhost:9200/[ES_INDEX]/_count)

e.g.
[http://localhost:9200/free-text-reports/_count](http://localhost:9200/free-text-reports/_count)